# Testing Alt Az conversion in decanO.py

In [46]:
# Ok, so one thing that I was actually missing was a UTC to Cairo time converter
# this works out to dt = 2 hr on Jan 1 2000; when implementing that the Alt/Az matched MUCH better 

# HOWEVER, no amount of dt hours on Jan 1 -1300 aligns the alt/az between Stellarium and Python; 
# see this stackexchange for partial discussion:
# https://astronomy.stackexchange.com/questions/27447/differences-between-alt-az-in-stellarium-and-astropy

# If we take the Stellarium data as gospel, one path forward I could see is to "rotate" the Python year
# and define a new "effective Jan 1" by when the stars behave the closest 
# maybe count up leap days?

# However, it's not clear to me that Stellarium assumptions > Python assumptions 
# Worth having a chat with Sarah to agree on a way forward before doing a lot of nonsense

In [47]:
### Stelarium azimuth

# double StelLocation::getAzimuthForLocation(double longObs, double latObs, double longTarget, double latTarget)
# {
# 	longObs    *= M_PI_180;
# 	latObs     *= M_PI_180;
# 	longTarget *= M_PI_180;
# 	latTarget  *= M_PI_180;

# 	double az = atan2(sin(longTarget-longObs), cos(latObs)*tan(latTarget)-sin(latObs)*cos(longTarget-longObs));
# 	if (StelApp::getInstance().getFlagSouthAzimuthUsage())
# 		az += M_PI;
# 	return StelUtils::fmodpos(M_180_PI * az, 360.0);
# }


# 

<!-- double StelLocation::getAzimuthForLocation(double longObs, double latObs, double longTarget, double latTarget)
{
	longObs    *= M_PI_180;
	latObs     *= M_PI_180;
	longTarget *= M_PI_180;
	latTarget  *= M_PI_180;

	double az = atan2(sin(longTarget-longObs), cos(latObs)*tan(latTarget)-sin(latObs)*cos(longTarget-longObs));
	if (StelApp::getInstance().getFlagSouthAzimuthUsage())
		az += M_PI;
	return StelUtils::fmodpos(M_180_PI * az, 360.0); -->

In [48]:
### Import Statements
import numpy as np
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun, Angle, Longitude
from sunpy.coordinates import frames, sun
import star_chart_spherical_projection as scsp
import csv
import argparse
import os
import errno

In [49]:
def precessedCoords(declist, year):
    '''
    A function to find the Dec and RA of list of decans accounting for the precession of the equinoxes.
    Inputs: 
        declist = list of named decans to querry
        year = 
    Outputs:
        obj_list = list of decans' RAs and Decs as an astropy SkyCoords object
        year = year BC for which to return data
        hd_list = a variable to set the header for the csv file
    '''
    # querry available stars from scsp
    # note: need to fix this for stars not listed by name in scsp!
    years_since = -2000 - int(year)
    #years_since = 0
    star_dict = scsp.finalPositionOfStars(declist, yearSince2000=years_since)
    star_names = star_dict.keys()

    # set up lists 
    obj_list = []
    hd_list = ["Julian Date", "Human Readable Date", "Sun Azimuth", "Sun Altitude"]

    # calculate objects
    for name in star_names:
        ra_temp = star_dict[name]["RA"].split(".")
        # Check RA transform?
        RA = Angle(ra_temp[0] + "h" + ra_temp[1] + "m" + ra_temp[2] + "s").deg
        dec_temp = star_dict[name]["Declination"]
        Dec = Angle(dec_temp, unit="deg").deg
        obj = SkyCoord(ra=RA, dec=Dec, unit="deg")
        obj_list.append(obj)
        hd_list.append(name + " Azimuth")
        hd_list.append(name + " Altitude")
    return(obj_list, hd_list)


In [50]:
# ##
# #### Parse the arguments
# ##


# parser = argparse.ArgumentParser('My program')
# parser.add_argument('-d','--decan', nargs='+', type=str)
# parser.add_argument('-yBC', '--yearBC')
# parser.add_argument('-m', '--month')

# args = parser.parse_args()

# decans = list(args.decan)
# year = str(args.yearBC)
# month = str(args.month)

decans = ["Sirius", "Saiph", "Rigel", "Hamal", "Rasalhague", "Sheratan"]
#decans = ["Hamal", "Rasalhague", "Rigel", "Saiph", "Sheratan","Sirius"]
year = "1300"
month = "01"

In [51]:

# ##
# #### Determining the when and where
# ##

# # Set Location on Earth 

Luxor = EarthLocation(lat=25.68*u.deg, lon=31.55*u.deg, height=89*u.m)


# # Length of month 

len31 = ['01', '03', '05', '07', '08', '10', '12']
len30 = ['02', '04', '06', '09', '11']

if month in len31:
	end = 31
else:
	end = 30

# # Times and dates

start = (Time('-0' + year + '-' + month + '-01T00:00:00.000').jd)
days = start + 1 * np.arange(0, 365) # iterate for a year 
#days = start + 1 * np.arange(0, 1) # iterate for a day to test 
dhour = 0.04166666674427688
d4min = 0.00277777784503996
hours = dhour * np.arange(0, 24)
minutes = d4min * np.arange(0, 15)


/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [64]:
jd = (Time("2000" + '-' + month + '-01T00:12:00.000').jd)
jdS = 2451544.91667 # in Stellarium, Jan 1 -1300 00:00:00 = 1246232.40934



jd - jdS

-0.4083366668783128

In [53]:
jd = (Time("1400" + '-' + month + '-01T00:00:00.000').jd)
jdS = 2232407.40933 # in Stellarium, Jan 1 -1300 00:00:00 = 1246232.40934


#0.09066999983042479
#-8.909330000169575
#-7.909330000169575


jd - jdS

-7.909330000169575

In [54]:
jd = (Time("0000" + '-' + month + '-01T00:00:00.000').jd)
jdS = 1721057.40933 # in Stellarium, Jan 1 -1300 00:00:00 = 1246232.40934

jd - jdS

2.0906700000632554

In [59]:
jd = (Time('-0' + "1400" + '-' + month + '-01T00:00:00.000').jd)
jdS = 1209707.40933 # in Stellarium, Jan 1 -1300 00:00:00 = 1246232.40934

jd - jdS
#da = jd - 2451545.0 # Julian centuries since 1 Jan 2000 at 12 UTC

print(jd)

1209720.5


/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [87]:
Luxor.lon.deg

31.550000000000004

In [95]:
days_offset_list = [-14,-14,-13,-12,-11,-11] # bring in line with Stellarium

year = 1100

if int(year) > 1499:
    dS_off = -14 
elif int(year) < 1201:    
    dS_off = -11
else:
    dS_off = 1 - (int(year) / 100 )\
    
print(dS_off)    

-11


In [88]:
years = ["1600", "1500", "1400", "1300", "1200", "1100"]
jdS_list = [1136657.40933, 1173182.40933, 1209707.40933, 1246232.40933, 1282757.40933, 1319282.40933]

for i in range(0, len(jdS_list)):
    jdS = jdS_list[i]
    jd = (Time('-0' + years[i] + '-' + month + '-01T00:00:00.000', scale="local", location = Luxor).jd) - (Luxor.lon.deg/15.0) * dhour
    print(round(jdS - jd))


-14
-14
-13
-12
-11
-11


In [78]:
jd = (Time('-0' + "1300" + '-' + month + '-01T00:00:00.000', scale="local", location = Luxor).jd)
jdS = 1246232.40933 # in Stellarium, Jan 1 -1300 00:00:00 = 1246232.40934

jd - jdS
#da = jd - 2451545.0 # Julian centuries since 1 Jan 2000 at 12 UTC

print(Time(jd - (32.6421/15.0) * dhour, format = "jd").isot)
print(Time(jdS , format="jd").isot)


-1301-12-31T21:49:25.896
-1301-12-19T21:49:26.112


/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [74]:
dhour

0.04166666674427688

In [33]:
jd = (Time('-0' + "1200" + '-' + month + '-01T00:00:00.000').jd)
jdS = 1282757.40933 # in Stellarium, Jan 1 -1300 00:00:00 = 1246232.40934

jd - jdS
#da = jd - 2451545.0 # Julian centuries since 1 Jan 2000 at 12 UTC

11.090670000063255

In [34]:
jd = (Time('-0' + "1100" + '-' + month + '-01T00:00:00.000').jd)
jdS = 1319282.40933 # in Stellarium, Jan 1 -1300 00:00:00 = 1246232.40934

jd - jdS
#da = jd - 2451545.0 # Julian centuries since 1 Jan 2000 at 12 UTC

/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


11.090670000063255

12.09065999998711

In [31]:
Time(1246232.40934 + 12.09065999998711, format="jd").isot

/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


'-1300-01-01T00:00:00.000'

In [14]:

# ###
# ##### Writing the .txt file
# ###
# direct = os.getcwd() # current working directory
# ditect = direct + '/DecanLists' # directory where the .txt files go

# if not os.path.exists(os.path.dirname(direct)):
#     try:
#         os.makedirs(os.path.dirname(direct))
#     except OSError as exc: # Guard against race condition
#         if exc.errno != errno.EEXIST:
#             raise

# # name the txt file
# # note to self: should I just make it a csv or something?

# filename = direct + "/" + "data" + month + year + "BC.txt"


In [16]:

## Get coordinates of decans while accounting for precession of the equinoxes

#obj = SkyCoord.from_name(decan)
(obj_list, hd_list) = precessedCoords(decans, year)



In [17]:
day = days[0]
hour = hours[0]
mins = minutes[0]
obj = obj_list[0]

temptime = day + hour + mins

# define alt and az frame (time and date)
frame_altaz = AltAz(obstime=Time(temptime, format = 'jd'), location=Luxor)

# Sun coords
c = SkyCoord(0 * u.arcsec, 0 * u.arcsec, obstime=Time(temptime, format = 'jd'), observer="earth", frame=frames.Helioprojective)
sun_altaz = c.transform_to(frame_altaz)

# star coords
c1 = SkyCoord(obj)
info_temp = c1.transform_to(frame_altaz)
az = '{0.az:.1f}'.format(info_temp)[0:-4]
alt = '{0.alt:.1f}'.format(info_temp)[0:-4]



/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "epv00" yielded 1 of "warning: date outsidethe range 1900-2100 AD"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utcut1" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [231]:
obj

<SkyCoord (ICRS): (ra, dec) in deg
    (348.87916667, 5.2974748)>

In [232]:
info_temp

<SkyCoord (AltAz: obstime=1246244.5, location=(4901637.66358938, 3009614.36089623, 2747193.79657619) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (17.45261761, -73.6225216)>

In [46]:
angtest = Angle("348.87916667", unit="deg")
angtest.to_string(unit=u.hour, decimal=True)

'23.2586'

In [35]:
info_temp = obj.transform_to(AltAz(location=Luxor))

AttributeError: 'NoneType' object has no attribute 'scale'

In [13]:
# FROM STELLARIUM on 1/1/-1300 (~00:00:56)
# Hamal (Alpha Aries)

# (J2000)   RA: 02h 07m 10s Dec: 23 deg 27' 44"
# (on date) RA: 23h 14m 56s Dec: 05 deg 39' 44"

# Az/Alt: 279 deg 07' 32" / -5 46' 30"


#Result from astrogreg
# Alt:-042° 03' 19.13"
# Az: +304° 52' 23.81"

In [17]:
print(obj_list[0])
Angle("23h14m56s").degree

<SkyCoord (ICRS): (ra, dec) in deg
    (348.87916667, 5.2974748)>


348.7333333333333

In [116]:
year = "2023"
month = "01"

#start = (Time('-0' + year + '-' + month + '-01T00:00:00.000').jd)
time_str = year + '-' + month + '-01T00:00:00.000'
start = Time(time_str).jd
days = start + 1 * np.arange(0, 365) # iterate for a year 
#days = start + 1 * np.arange(0, 1) # iterate for a day to test 
dhour = 0.04166666674427688
d4min = 0.00277777784503996
hours = dhour * np.arange(0, 24)
minutes = d4min * np.arange(0, 15)

In [117]:
time_str

'2023-01-01T00:00:00.000'

In [118]:
time = '2023-01-01T00:00:00.00'
t = Time(time, format='fits', scale = 'utc').jd
t2 = Time(time).jd


In [119]:
time_str

'2023-01-01T00:00:00.000'

In [108]:
t2

2460093.5

In [73]:
temptime

982174.5

In [202]:
start = (Time('2000-01-01T00:00:00.000', format='fits', scale = 'utc').jd)
end = (Time('2000-01-01T02:00:00.000', format='fits', scale = 'utc').jd)


In [205]:
end - start

0.08333333348855376

In [233]:
# Do they match today?

# Stellarium says at 2000-01-01 00:00:30

# Ra/DEc 2h07m11.5s / 23deg27'46"
#
# Az/Alt 284deg45'32" / 26 deg 22' 45"

year = "1300"
month = "01"

#start = (Time('-0' + year + '-' + month + '-01T00:00:00.000', format='fits', scale = 'utc').jd)
start = (Time('-0' + year + '-' + month + '-01T00:00:00.000', scale = 'utc').jd)
days = start + 1 * np.arange(0, 365) # iterate for a year 
#days = start + 1 * np.arange(0, 1) # iterate for a day to test 
dhour = 0.04166666674427688
dtzone = dhour * 2 # 2 hours diff between UTC to Egypt time
d4min = 0.00277777784503996
hours = dhour * np.arange(0, 24)
minutes = d4min * np.arange(0, 15)

(obj_list, hd_list) = precessedCoords(decans, year)


/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [278]:

day = days[0]
hour = hours[0]
mins = minutes[0]
obj = obj_list[0]

temptime = day + hour + mins

oTime = Time(temptime, format = 'jd') + 5.6 * dhour
frame_altaz = AltAz(obstime=oTime, location=Luxor)

# Sun coords
c = SkyCoord(0 * u.arcsec, 0 * u.arcsec, obstime=oTime, observer="earth", frame=frames.Helioprojective)
sun_altaz = c.transform_to(frame_altaz)
# star coords
info_temp = obj.transform_to(frame_altaz)
az = '{0.az:.1f}'.format(info_temp)[0:-4]
alt = '{0.alt:.1f}'.format(info_temp)[0:-4]

/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "epv00" yielded 1 of "warning: date outsidethe range 1900-2100 AD"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utcut1" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [279]:
print(obj)
print(info_temp)
# 279 -5

# test_ang = Angle(276.14507468, unit = "deg")
# test_ang.to_string(unit=u.hourangle)

<SkyCoord (ICRS): (ra, dec) in deg
    (348.87916667, 5.2974748)>
<SkyCoord (AltAz: obstime=1246244.7333333339, location=(4901637.66358938, 3009614.36089623, 2747193.79657619) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (98.65705433, -5.0598365)>


In [241]:
test_ang = Angle(348.87916667, unit = "deg")
print(test_ang.to_string(unit=u.hourangle))

23h15m31.0000008s


In [222]:

ctest = SkyCoord(ra=31.79166667*u.deg, dec=23.26999508*u.deg)
frame_altaz = AltAz(obstime=Time(temptime, format = 'jd'), location=Luxor)
info_temp = ctest.transform_to(frame_altaz)
print(info_temp)



<SkyCoord (AltAz: obstime=1246244.5, location=(4901637.66358938, 3009614.36089623, 2747193.79657619) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (305.17560816, -42.05330284)>


'3h23m12s'

In [198]:
from zoneinfo import ZoneInfo

In [199]:
import ephem

d = ephem.Date('-1300/01/01 00:00:00')

sun = ephem.Sun()
sun.compute(d)
print(sun.ra, sun.dec)

Lux = ephem.Observer()
Lux.date = d
Lux.lat = '25.68'
Lux.lon = '31.55'

print(Lux.next_rising(sun))

print(sun.az, sun.alt)

18:00:52.42 -23:50:51.9
-1300/1/1 04:45:17
116:10:50.3 -0:16:12.5


In [136]:
temptime

2451544.5

In [110]:
test_ang = Angle(32.11666667, unit = "deg")
test_ang.to_string(unit=u.hourangle)

'2h08m28.0000008s'

In [38]:
tt = Time(temptime, format = 'jd')

tt.fits

/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "d2dtf" yielded 1 of "dubious year (Note 5)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


'-01300-01-01T00:00:00.000'

In [39]:
obj.transform_to(AltAz(obstime=tt, location=Luxor))

/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utcut1" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/usr/local/anaconda3/envs/decanOpy/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "epv00" yielded 1 of "warning: date outsidethe range 1900-2100 AD"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


<SkyCoord (AltAz: obstime=1246244.5, location=(4901637.66358938, 3009614.36089623, 2747193.79657619) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (17.45261761, -73.6225216)>

In [11]:
print(alt, az)

-73.6 17.5


In [32]:

observing_location = EarthLocation(lat='52.2532', lon='351.63910339111703', height=100*u.m)  
observing_time = Time('2017-02-05 20:12:18')  
aa = AltAz(location=Luxor, obstime=observing_time)

coord = SkyCoord('4h42m', '-38d6m50.8s')
coord.transform_to(aa)

<SkyCoord (AltAz: obstime=2017-02-05 20:12:18.000, location=(4901637.66358938, 3009614.36089623, 2747193.79657619) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (211.85156395, 15.95706777)>

In [ ]:

# start writing the file

with open(filename, "w", newline='') as file:
    writer = csv.writer(file, delimiter='|')
    #writer.writerow(["Object: " + str(decans)])
    #writer.writerow(["Luxor = EarthLocation(lat=25.68*u.deg, lon=31.55*u.deg, height=89*u.m)"])
    #writer.writerow(["\n"])
    writer.writerow(hd_list) # write headers
    for day in days:
        for hour in hours:
            for mins in minutes:
                temptime = day + hour + mins
                # Sun coords
                c = SkyCoord(0 * u.arcsec, 0 * u.arcsec, obstime=Time(temptime, format = 'jd'), observer="earth", frame=frames.Helioprojective)
                frame_altaz = AltAz(obstime=Time(temptime, format = 'jd'), location=Luxor)
                sun_altaz = c.transform_to(frame_altaz)
                # decan coords
                info = ['{0:.16f}'.format(np.round(temptime, 10)),
                                 str(Time(temptime, format = 'jd').fits),
                                 '{0:.1f}'.format(sun_altaz.T.az)[0:-4], # [0:-4] get rid of trailing " deg" for later analysis
                                 '{0:.1f}'.format(sun_altaz.T.alt)[0:-4]]
                for obj in obj_list:
                    info_temp = obj.transform_to(AltAz(obstime=Time(temptime, format = 'jd'), location=Luxor))
                    info.append('{0.az:.1f}'.format(info_temp)[0:-4]) 
                    info.append('{0.alt:.1f}'.format(info_temp)[0:-4]) 
                # Write to file
                writer.writerow(info)



# ##
# #### Functions for data processing
# ##


# Import Decan Data

def ImportDecanData(direct, filename):
    
    '''
    A function to import data from a decanOpy-generated .txt file. 
    Inputs: 
        direct = string with the directory where the .txt file is located
        filename = string with name of file (name + month + year)
    Outputs:
        jd = Julian date
        date = human readable date
        DecAz = the azimuth of the decan
        DecAlt = the altitude of the decan
        SunAz = the azimuth of the Sun
        SunAlt = the altitude of the Sun
    '''
    
    jd = []
    date = []
    DecAz = []
    DecAlt = []
    SunAz = []
    SunAlt = []
    # Import Single Object
    with open(direct + filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        decan = next(csv_reader)[0]
        location = next(csv_reader)[0]
        trash = next(csv_reader)
        headers = next(csv_reader)
        for row in csv_reader:
            # time info
            jd.append(float(row[0]))
            date.append(row[1])
            # decan info
            DecAz.append(float(row[2][0:-4]))
            DecAlt.append(float(row[3][0:-4]))
            # solar info
            SunAz.append(float(row[4][0:-4]))
            SunAlt.append(float(row[5][0:-4]))
    return(jd, date, DecAz, DecAlt, SunAz, SunAlt)

def JustDecanData(direct, filename):
    
    '''
    A function to import just the can data data from a decanOpy-generated .txt file. 
    Used for the MaxMinAltAz function.
    Inputs: 
        direct = string with the directory where the .txt file is located
        filename = string with name of file (name + month + year)
    Outputs:
        DecAz = the azimuth of the decan
        DecAlt = the altitude of the decan
    '''
    
    DecAz = []
    DecAlt = []
    # Import Single Object
    with open(direct + filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter='|')
        decan = next(csv_reader)[0]
        location = next(csv_reader)[0]
        trash = next(csv_reader)
        headers = next(csv_reader)
        for row in csv_reader:
            # decan info
            DecAz.append(float(row[2][0:-4]))
            DecAlt.append(float(row[3][0:-4]))
            # solar info
    return(DecAz, DecAlt)

# Get Sunset and Sunrise Times

def SunRiseSet(jd, SunAlt):
    
    '''
    A function to create a list of indices where the Sun rises and sets in a given year. 
    This is useful for making sure we're tracking nightly, visible motion of the decans.
    Inputs: 
        jd = Julian date
        SunAlt = the altitude of the Sun
    Outputs:
        sunriseset = indices of sunrize and sunset in the jd & date columns
    '''
    
    sunriseset = []
    for i in range(360, len(jd), 360):
        temp = []
        for j in range(i - 360, i):
            if SunAlt[j] <= 0.4 and SunAlt[j] >= -0.4:
                if len(temp) == 0: 
                    temp.append(j)
                elif temp[-1] != j - 1:
                    temp.append(j)
        sunriseset.append(temp)
    return sunriseset

# Maximum and Minimum Nightly Altitude of Object

def MaxMinAltAz(direct, filename, jd, sunriseset):
    
    '''
    A function to create lists of minimum and maximum azimuths and altitudes of the decan. 
    This is useful for making sure we're tracking nightly, visible motion of the decans.
    Inputs: 
        direct = string with the directory where the .txt file is located
        filename = string with name of file (name + month + year)
        jv = Julian date
        sunriseset = indices of sunrize and sunset in the jd & date columns
    Outputs:
        sunriseset = indices of sunrize and sunset in the jd & date columns
        days = list of indices when it's daylight 
        minaz, maxaz = minimum and maximum azimuths of the decan per night
        minalt, maxalt = minimum and maximum altitudes of the decan per night
        riseaz, setaz = azimuth of decan at rise & set
        risealt, setalt = altitude of decan at rise & set
    '''
    
    (DecAz, DecAlt) = JustDecanData(direct, filename)
    maxalt = []
    minalt = []
    maxaz = []
    minaz = []
    riseaz = []
    setaz = []
    risealt = []
    setalt = []
    days = []
    for i in range(0, int(len(jd)/360) - 2):
        sset = sunriseset[i][1]
        srise = sunriseset[i + 1][0]
        maxalt.append(max(DecAlt[sset:srise]))
        minalt.append(min(DecAlt[sset:srise]))
        maxaz.append(max(DecAz[sset:srise]))
        minaz.append(min(DecAz[sset:srise]))
        riseaz.append(DecAz[srise])
        setaz.append(DecAz[sset])
        risealt.append(DecAlt[srise])
        setalt.append(DecAlt[sset])
        days.append(DecAlt[srise:sset])
    return(days, minaz, maxaz, minalt, maxalt, riseaz, setaz, risealt, setalt)

